In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ritikmittal20","key":"25d3d0e51c912ae6c69a1752d2f78b4d"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv /content/kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 64% 52.0M/80.9M [00:00<00:00, 284MB/s]
100% 80.9M/80.9M [00:00<00:00, 299MB/s]


In [ ]:
!unzip /content/sentiment140.zip

Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import os
import contractions
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
import pickle
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet as wn
import numpy as np

In [ ]:
from keras.layers import TextVectorization
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
import tensorflow as tf

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding="latin-1", header=None)

In [ ]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.rename(columns={0:"target", 1:"ids", 2:"date", 3:"flag", 4:"user", 5:"text"}, inplace=True)

In [ ]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.drop(labels=["ids","date","flag","user"], axis=1, inplace= True)

In [ ]:
data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data["target"].unique()

array([0, 4])

In [ ]:
data["text"].iloc[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

# Text Normalization

In [ ]:
def text_normalization(text):

  return text.lower()

In [ ]:
os.cpu_count()

2

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(text_normalization, list(data["text"])))

In [ ]:
data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Expand Contractions

In [ ]:
def expand_contractions(text):

  return contractions.fix(text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(expand_contractions, list(data["text"])))

In [ ]:
data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he cannot update his facebook by...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it is not behaving at all..."


In [ ]:
data["text"].iloc[0]

'@switchfoot http://twitpic.com/2y1zl - awww, that is a bummer.  you shoulda got david carr of third day to do it. ;d'

# Noisy Token removel using Regression Expression

In [ ]:
noisy_tokens_regex = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/\/{2}[\d\w-]+(\.[\d\w-]+)*(?:\/(?:[^\s\/]*))*|\W+[\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'


In [ ]:
def noisy_tokens_removel(text):

  return re.sub(pattern=noisy_tokens_regex,repl=" ", string=text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(noisy_tokens_removel, list(data["text"])))

In [ ]:
data.head()

In [ ]:
data["text"].iloc[0]

In [ ]:
leftover_noisy_tokens_regex = r'\b\w\b|[^\u0000-\u007f]+|_+|\W+'
# it is used here to remove some extra noisy tokens which is currently present in data

In [ ]:
def leftover_noisy_tokens_removal(text):

  return re.sub(pattern=leftover_noisy_tokens_regex,repl=" ",string=text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(leftover_noisy_tokens_removal,list(data["text"])))

In [ ]:
data.head()

In [ ]:
data["text"].iloc[0]

In [ ]:
def tokenize_text(text):

  return word_tokenize(text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(tokenize_text,list(data["text"])))

In [ ]:
data.head()

# Remove Stop Words

In [ ]:
len(stopwords.words("english"))

In [ ]:
len(STOP_WORDS)

In [ ]:
all_stopwords = list(STOP_WORDS.union(set(stopwords.words("english"))))

In [ ]:
len(all_stopwords)

In [ ]:
def is_stopword(token):

  return not(token in all_stopwords)

In [ ]:
def remove_stopwords(tokenized_text):

  return list(filter(is_stopword,tokenized_text))

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  data["text"] = list(pool.map(remove_stopwords,list(data["text"])))

In [ ]:
!cp /content/drive/MyDrive/stp_words_removed_tokenized.pkl ./

In [ ]:
with open("/content/stp_words_removed_tokenized.pkl","rb") as file_handle:

  data = pickle.load(file_handle)

In [ ]:
data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, got,..."
1,0,"[upset, update, facebook, texting, cry, result..."
2,0,"[managed, save, rest, bounds]"
3,0,"[body, feels, itchy, like, fire]"
4,0,"[behaving, mad]"


In [ ]:
def get_wordnet_pos_tag(treebank_tagged_token):

  if treebank_tagged_token[1].startswith("J"):
    return (treebank_tagged_token[0], wn.ADJ)

  elif treebank_tagged_token[1].startswith("V"):
    return(treebank_tagged_token[0], wn.VERB)

  elif treebank_tagged_token[1].startswith("N"):
    return(treebank_tagged_token[0], wn.NOUN)

  elif treebank_tagged_token[1].startswith("R"):
    return(treebank_tagged_token[0], wn.ADV)

  else:
    return(treebank_tagged_token[0], wn.NOUN)

In [ ]:
def get_pos_tag(tokenized_text):

  return list(map(get_wordnet_pos_tag, pos_tag(tokenized_text)))

In [ ]:
data["text"] = list(map(get_pos_tag,list(data["text"])))

In [ ]:
data.head()

,target,text
0,0,"[(twitpic, n), (com, n), (zl, n), (awww, n), (..."
1,0,"[(upset, a), (update, a), (facebook, n), (text..."
2,0,"[(managed, v), (save, v), (rest, n), (bounds, n)]"
3,0,"[(body, n), (feels, n), (itchy, v), (like, n),..."
4,0,"[(behaving, v), (mad, n)]"


# Use Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_token(token_pos_tuple):

  if token_pos_tuple == None:
    return ""
  else:
    return lemmatizer.lemmatize(word=token_pos_tuple[0], pos=token_pos_tuple[1])

In [ ]:
def lemmatize_text(tokenized_text):

  if len(tokenized_text) > 0:
    return list(map(lemmatize_token,tokenized_text))
  else:
    return[""]

In [ ]:
data["text"] = list(map(lemmatize_text,list(data["text"])))

In [ ]:
!cp /content/drive/MyDrive/preprocessed_tweets.pkl ./

In [ ]:
with open("/content/preprocessed_tweets.pkl","rb") as file_handle:

  data = pickle.load(file_handle)

In [ ]:
data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
1,0,"[upset, update, facebook, texting, cry, result..."
2,0,"[manage, save, rest, bound]"
3,0,"[body, feel, itchy, like, fire]"
4,0,"[behave, mad]"


In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  max_sequence_len = max(list(pool.map(lambda x: len(x), list(data["text"]))))

In [ ]:
max_sequence_len

50

In [ ]:
min_tokens_thresh = 10

In [ ]:
reduced_data = data[data['text'].apply(len) > min_tokens_thresh]

In [ ]:
reduced_data

,target,text
35,0,"[ok, sick, spend, hour, sit, shower, sick, sta..."
44,0,"[fall, asleep, hear, tracy, girl, body, find, ..."
46,0,"[check, user, timeline, blackberry, look, like..."
50,0,"[broadband, plan, massive, broken, promise, ww..."
57,0,"[sad, feeling, dallas, go, get, think, show, u..."
...,...,...
1599950,4,"[omg, good, ben, jerry, cookie, dough, icecrea..."
1599951,4,"[oooo, haha, wake, ready, eat, delicious, brea..."
1599965,4,"[thanks, follow, nice, website, check, carriag..."
1599966,4,"[get, home, hour, ago, ate, lunch, watch, tv, ..."


In [ ]:
reduced_data["text"] = reduced_data["text"].apply(lambda x: " ".join(x))

<ipython-input-16-9e5e8e4a7c05>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_data["text"] = reduced_data["text"].apply(lambda x: " ".join(x))


In [ ]:
reduced_data.head()

,target,text
35,0,ok sick spend hour sit shower sick stand hold ...
44,0,fall asleep hear tracy girl body find sad hear...
46,0,check user timeline blackberry look like twank...
50,0,broadband plan massive broken promise www diig...
57,0,sad feeling dallas go get think show use music...


In [ ]:
# all_tokens = [token for tokens_list in data['text'] for token in tokens_list]

# unique_words = set(all_tokens)

# vocabulary_size = len(unique_words)

In [ ]:
max_vocabulary_size = 40000

In [ ]:
vectorization_layer = TextVectorization(max_tokens= max_vocabulary_size, output_sequence_length= max_sequence_len)

In [ ]:
vectorization_layer.adapt(data= np.array(reduced_data["text"]))

In [ ]:
def vectorize_tweet(raw_text):

  return vectorization_layer(raw_text).numpy()

In [ ]:
vectorized_tweets = list(reduced_data["text"].apply(vectorize_tweet))

In [ ]:
vectorized_tweets

[array([  88,  103,  283,   51,  145,  279,  103,  580,  469, 2769,    8,
        3495,   45,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]),
 array([ 272,  330,  227, 5967,   76,  421,   47,   50,  288,  119,  157,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]),
 array([  131,  1245,  5808,  1277,    22,     8,     1,   241,    37,
         2429, 14134,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,    

In [ ]:
reduced_data_np_array = np.array(vectorized_tweets)

In [ ]:
reduced_data_np_array.shape

(146954, 50)

In [ ]:
reduced_data_np_array

array([[  88,  103,  283, ...,    0,    0,    0],
       [ 272,  330,  227, ...,    0,    0,    0],
       [ 131, 1245, 5808, ...,    0,    0,    0],
       ...,
       [  73,  112,   56, ...,    0,    0,    0],
       [   6,   17,   51, ...,    0,    0,    0],
       [ 959, 1719,  355, ...,    0,    0,    0]])

In [ ]:
target_np_array = np.array(reduced_data["target"]).reshape(reduced_data["target"].shape[0],1)

In [ ]:
target_np_array.shape

(146954, 1)

In [ ]:
target_np_array

array([[0],
       [0],
       [0],
       ...,
       [4],
       [4],
       [4]])

In [ ]:
reduced_data_np_array = np.concatenate((reduced_data_np_array,target_np_array),axis=1)

In [ ]:
reduced_data_np_array.shape

(146954, 51)

In [ ]:
reduced_data_np_array

array([[  88,  103,  283, ...,    0,    0,    0],
       [ 272,  330,  227, ...,    0,    0,    0],
       [ 131, 1245, 5808, ...,    0,    0,    0],
       ...,
       [  73,  112,   56, ...,    0,    0,    4],
       [   6,   17,   51, ...,    0,    0,    4],
       [ 959, 1719,  355, ...,    0,    0,    4]])

In [ ]:
!cp /content/drive/MyDrive/vectorized_data.npz ./

In [ ]:
reduced_data_np_array = np.load("/content/vectorized_data.npz")

In [ ]:
for key in reduced_data_np_array:
  vectorized_data = reduced_data_np_array[key]

In [ ]:
vectorized_data.shape

(146954, 51)

In [ ]:
vectorized_data

array([[  88,  103,  283, ...,    0,    0,    0],
       [ 272,  330,  227, ...,    0,    0,    0],
       [ 131, 1245, 5808, ...,    0,    0,    0],
       ...,
       [  73,  112,   56, ...,    0,    0,    4],
       [   6,   17,   51, ...,    0,    0,    4],
       [ 959, 1719,  355, ...,    0,    0,    4]])

In [ ]:
vectorized_data[:,50] = vectorized_data[:,50]//4   # Scale all the positive value 4 into 1

In [ ]:
vectorized_data

array([[  88,  103,  283, ...,    0,    0,    0],
       [ 272,  330,  227, ...,    0,    0,    0],
       [ 131, 1245, 5808, ...,    0,    0,    0],
       ...,
       [  73,  112,   56, ...,    0,    0,    1],
       [   6,   17,   51, ...,    0,    0,    1],
       [ 959, 1719,  355, ...,    0,    0,    1]])

In [ ]:
pos_vectorized_data = vectorized_data[vectorized_data[:,50] == 0]

In [ ]:
neg_vectorized_data = vectorized_data[vectorized_data[:,50] == 1]

In [ ]:
pos_train_vectorized_data = pos_vectorized_data[0:70000]
pos_test_vectorized_data = pos_vectorized_data[70000:]

In [ ]:
neg_train_vectorized_data = neg_vectorized_data[0:70000]
neg_test_vectorized_data = neg_vectorized_data[70000:]

In [ ]:
train_vectorized_data = np.concatenate((neg_train_vectorized_data,pos_train_vectorized_data),axis=0)

In [ ]:
train_vectorized_data.shape

(140000, 51)

In [ ]:
train_vectorized_data

array([[  225,     3,    54, ...,     0,     0,     1],
       [   16,  6383,   160, ...,     0,     0,     1],
       [  278,   324, 10120, ...,     0,     0,     1],
       ...,
       [   37,    46,   620, ...,     0,     0,     0],
       [   85,   100,    13, ...,     0,     0,     0],
       [  134,   925,   523, ...,     0,     0,     0]])

In [ ]:
test_vectorized_data = np.concatenate((neg_test_vectorized_data,pos_test_vectorized_data),axis=0)

In [ ]:
test_vectorized_data.shape

(6954, 51)

In [ ]:
train_vectorized_data

array([[  225,     3,    54, ...,     0,     0,     1],
       [   16,  6383,   160, ...,     0,     0,     1],
       [  278,   324, 10120, ...,     0,     0,     1],
       ...,
       [   37,    46,   620, ...,     0,     0,     0],
       [   85,   100,    13, ...,     0,     0,     0],
       [  134,   925,   523, ...,     0,     0,     0]])

In [ ]:
train_mb_size = 1400
num_epochs = 5
embedding_dim = 64

In [ ]:
train_X_vectorized_data = train_vectorized_data[:,0:50]
train_Y_vectorized_data = train_vectorized_data[:,50]

test_X_vectorized_data = test_vectorized_data[:,0:50]
test_Y_vectorized_data = test_vectorized_data[:,50]

In [ ]:
train_Y_vectorized_data = train_Y_vectorized_data.reshape(train_Y_vectorized_data.shape[0],1)
test_Y_vectorized_data = test_Y_vectorized_data.reshape(test_Y_vectorized_data.shape[0],1)

In [ ]:
def training_datagen():

  for _ in range(num_epochs):

    for i in range(train_vectorized_data.shape[0]//train_mb_size):

      train_X_mb = train_X_vectorized_data[i*train_mb_size:(i+1)*train_mb_size,:]
      train_Y_mb = train_Y_vectorized_data[i*train_mb_size:(i+1)*train_mb_size,:]

      yield train_X_mb, train_Y_mb

In [ ]:
train_datagen = training_datagen()

In [ ]:
train_datagen

<generator object training_datagen at 0x7f53f2804900>

In [ ]:
train_X_mb, train_Y_mb = train_datagen.__next__()

In [ ]:
train_X_mb.shape

(1400, 50)

In [ ]:
train_Y_mb.shape

(1400, 1)

In [ ]:
test_mb_size = 61

In [ ]:
def testing_datagen():

  for _ in range(num_epochs):

    for i in range(test_vectorized_data.shape[0]//test_mb_size):

      test_X_mb = test_X_vectorized_data[i*test_mb_size:(i+1)*test_mb_size,:]
      test_Y_mb = test_Y_vectorized_data[i*test_mb_size:(i+1)*test_mb_size,:]

      yield test_X_mb, test_Y_mb

In [ ]:
test_datagen = testing_datagen()

In [ ]:
test_datagen

<generator object testing_datagen at 0x7f53bcb60900>

In [ ]:
test_X_mb, test_Y_mb = train_datagen.__next__()

In [ ]:
test_X_mb.shape

(1400, 50)

In [ ]:
test_Y_mb.shape

(1400, 1)

In [ ]:
def create_bin_class_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,),dtype="int64"))
  rnn_model.add(Embedding(input_dim=max_vocabulary_size,output_dim=embedding_dim,
                          input_length=50))
  rnn_model.add(LSTM(units=embedding_dim))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [ ]:
rnn_model = create_bin_class_rnn()

In [ ]:
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          2560000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,593,089
Trainable params: 2,593,089
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_model.compile(loss="binary_crossentropy",metrics=["Accuracy",
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall()])

In [ ]:
rnn_model.fit(training_datagen(),
              epochs=num_epochs,
              #validation_data=(test_X_vectorized_data,test_Y_vectorized_data),
              validation_data = testing_datagen(),
              steps_per_epoch=100, validation_steps=114,)

Epoch 1/5
100/100 [==============================] - 72s 687ms/step - loss: 0.3224 - Accuracy: 0.8900 - precision_2: 0.8197 - recall_2: 1.0000 - val_loss: 0.1090 - val_Accuracy: 0.9779 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/5
100/100 [==============================] - 68s 681ms/step - loss: 0.3711 - Accuracy: 0.8500 - precision_2: 0.8571 - recall_2: 0.8400 - val_loss: 0.1143 - val_Accuracy: 0.9779 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/5
100/100 [==============================] - 63s 628ms/step - loss: 0.3002 - Accuracy: 0.9000 - precision_2: 0.9000 - recall_2: 0.9000 - val_loss: 0.1203 - val_Accuracy: 0.9779 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 4/5
100/100 [==============================] - 64s 635ms/step - loss: 0.2577 - Accuracy: 0.9400 - precision_2: 0.9400 - recall_2: 0.9400 - val_loss: 0.1255 - val_Accuracy: 0.9779 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 5/5
100/100 [=============

100/100 [==============================] - 66s 658ms/step - loss: 0.2377 - Accuracy: 0.9400 - precision_2: 0.9400 - recall_2: 0.9400 - val_loss: 0.0035 - val_Accuracy: 1.0000 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
